In [1]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import LSTM
from keras.initializers import RandomNormal
from keras.callbacks import ModelCheckpoint, EarlyStopping
import json
import csv
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
file_path = "./singlerack_all.csv"
server_list = []
conditioner_outlet_temp = []
conditioner_inlet_temp = []


timestep = 36
predict_horizon = 12

class Server(object):
    def __init__(self, i):
        self.id = i
        self.inlet_temp = []
        self.outlet_temp = []
        self.cpu = []
        self.memory = []


def strided_app(a, L, S):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size - L) // S) + 1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows, L), strides=(S * n, n), writeable=False)


def process_data():
    for i in range(15):
        server_list.append(Server(i))
    with open(file_path, "r", encoding='utf-8') as datacsv:
        csvr = csv.reader(datacsv)
        for row in csvr:
            i = 1
            for server in server_list:
                server.outlet_temp.append(float(row[i]))
                i = i + 1
            for server in server_list:
                server.inlet_temp.append(float(row[46 - i]))
                i = i + 1
            conditioner_outlet_temp.append(float(row[i]))
            i = i + 1
            conditioner_inlet_temp.append(float(row[i]))
            i = i + 6
            for server in server_list:
                # if(server.id<10):
                #     server.cpu.append(float(row[i])/10)
                # else:
                #     server.cpu.append(float(row[i]))
                server.cpu.append(float(row[i]) / 100)
                i = i + 6

process_data()



In [2]:
%matplotlib qt5
from keras.models import load_model
import matplotlib.pyplot as plt
i=0
for server in server_list:
    i+=1
    plt.plot(np.arange(len(server.inlet_temp)),server.inlet_temp,label=str(i))
plt.plot(np.arange(len(conditioner_inlet_temp)),conditioner_inlet_temp,label="c_in")
plt.plot(np.arange(len(conditioner_outlet_temp)),conditioner_outlet_temp,label="c_out")
plt.legend()
plt.show()



In [3]:
temp_train_x = []
temp_train_y = []
server=server_list[13]
temp = np.array(server.inlet_temp)
data = strided_app(temp, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
y = data[:, -1]
if isinstance(temp_train_x, list):
    temp_train_x = x
    temp_train_y = y
else:
    temp_train_x = np.concatenate((temp_train_x, x), axis=0)
    temp_train_y = np.concatenate((temp_train_y, y), axis=0)
cpu_x = []

cpu = np.array(server.cpu)
data = strided_app(cpu, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
if isinstance(cpu_x, list):
    cpu_x = x
else:
    cpu_x = np.concatenate((cpu_x, x), axis=0)

cpu_pre_model = load_model("./model/predict_cpu_ts36_ph12_0.19.hdf5")
cpu_pre_result = cpu_pre_model.predict(cpu_x)
mixed_train_x = np.concatenate((temp_train_x, cpu_pre_result), axis=1)
temp_pre_model=load_model("./model/predict_temp_usingcpu_ts36_ph12_0.19.hdf5")
temp_pre_res=temp_pre_model.predict(mixed_train_x)


In [4]:
temp_cpu_combined_x=np.concatenate((temp_train_x,cpu_x),axis=1)
temp_cpu_combined_model=load_model("./model/predict_temp_combined_cpu+temp_ts36_ph12_0.24.hdf5")
temp_cpu_combined_y=temp_cpu_combined_model.predict(temp_cpu_combined_x)


In [5]:
%matplotlib qt5
plt.plot(np.arange(len(server.inlet_temp[2736:3236])),server.inlet_temp[2736:3236],label='origin')
plt.plot(np.arange(len(temp_pre_res[2700:3200])),temp_pre_res[2700:3200],label='double lstm')
plt.plot(np.arange(len(temp_cpu_combined_y[2700:3200])),temp_cpu_combined_y[2700:3200],label='single lstm')
plt.legend()
plt.show()


In [6]:
temp_train_x = []
temp_train_y = []

temp = np.array(server.inlet_temp)
data = strided_app(temp, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
y = data[:, -1]
t_conditioner_outlet_temp = np.array(conditioner_outlet_temp)[48:]
t_conditioner_outlet_temp = t_conditioner_outlet_temp.reshape(-1, 1)
x = np.concatenate((x, t_conditioner_outlet_temp), axis=1)
if isinstance(temp_train_x, list):
    temp_train_x = x
    temp_train_y = y
else:
    temp_train_x = np.concatenate((temp_train_x, x), axis=0)
    temp_train_y = np.concatenate((temp_train_y, y), axis=0)
cpu_x = []

cpu = np.array(server.cpu)
data = strided_app(cpu, timestep + predict_horizon + 1, 1)
x = data[:, :-1 - predict_horizon]
if isinstance(cpu_x, list):
    cpu_x = x
else:
    cpu_x = np.concatenate((cpu_x, x), axis=0)

# cpu_pre_model = train_cpu_model(timestep, predict_horizon)
from keras.models import load_model
cpu_pre_model = load_model("./model/predict_cpu_ts36_ph12_0.19.hdf5")
cpu_pre_result = cpu_pre_model.predict(cpu_x)
mixed_train_x = np.concatenate((temp_train_x, cpu_pre_result), axis=1)
temp_pre_model_cpu_conditon=load_model("./model/predict_temp_usingcpu+condition_ts36_ph12_0.16.hdf5")

temp_pre_model_cpu_y=temp_pre_model_cpu_conditon.predict(mixed_train_x)

In [12]:
%matplotlib qt5
plt.plot(np.arange(len(server.inlet_temp[2836:3436])),server.inlet_temp[2836:3436],label='origin')
plt.plot(np.arange(len(temp_pre_model_cpu_y[2800:3400])),temp_pre_model_cpu_y[2800:3400],label='double lstm')
plt.plot(np.arange(len(temp_cpu_combined_y[2800:3400])),temp_cpu_combined_y[2800:3400],label='single lstm')
plt.xlabel("time(s)")
plt.ylabel("temperature(°C)")
plt.legend()
plt.show()

